In [60]:
import json
from pprint import pprint
import random
import spacy
from spacy.util import minibatch, compounding

In [44]:
train_file = open('training/json/agathemerus.json')
data = json.load(train_file)
random.shuffle(data)
pprint(data[:10])

[{'id': 0,
  'paragraphs': [{'sentences': [{'tokens': [{'ner': 'U-LOC',
                                             'orth': 'Ἀττικῆς,',
                                             'tag': '-'}]}]}]},
 {'id': 0,
  'paragraphs': [{'sentences': [{'tokens': [{'ner': 'O',
                                             'orth': 'ἡ',
                                             'tag': '-'}]}]}]},
 {'id': 0,
  'paragraphs': [{'sentences': [{'tokens': [{'ner': 'O',
                                             'orth': 'εἰς',
                                             'tag': '-'}]}]}]},
 {'id': 0,
  'paragraphs': [{'sentences': [{'tokens': [{'ner': 'U-LOC',
                                             'orth': 'πελάγει',
                                             'tag': '-'}]}]}]},
 {'id': 0,
  'paragraphs': [{'sentences': [{'tokens': [{'ner': 'O',
                                             'orth': 'ὧν',
                                             'tag': '-'}]}]}]},
 {'id': 0,
  'paragraphs':

In [40]:
train_data = data[:1500]
test_data = data[1500:]

In [56]:
# creates a blank model of greek, it doesn't recognise the ancient greek code grc
nlp = spacy.blank('el')
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner)

In [57]:
ner.add_label('U-LOC')
ner.add_label('U-PER')

In [58]:
# initialises a pip for training
# nlp.vocab.vectors.name = 'prosopography_pretrained_vectors'
optimizer = nlp.begin_training()

In [59]:
print(nlp.pipe_names)

['ner']


In [80]:
for i in range(1):
    print('Iteration #', i)
    
    random.shuffle(train_data)
    losses = {}
    
    for item in train_data:
        tokens = item['paragraphs'][0]['sentences'][0]['tokens'][0]
        # print(tokens)
        
        texts = tokens['orth']
        entity = tokens['ner']
        
        if entity == 'O':
            annotations = {'entities': []}
        else:
            annotations = {'entities': [(0, len(texts), tokens['ner'])]}
            
        # print(texts, annotations)
        
        nlp.update([texts], [annotations], sgd=optimizer, losses=losses)
    
    print('Losses', losses)

Iteration # 0
Losses {'ner': 392.379576995315}


In [81]:
# test the trained model
for item in test_data:
    tokens = item['paragraphs'][0]['sentences'][0]['tokens'][0]
    text = tokens['orth']
    expected = tokens['ner']
    
    doc = nlp(text)
    for ent in doc.ents:
        print('Entity in', text, ':', ent.label_, '| expected: ', expected)

Entity in Βοιωτίαν : U-LOC | expected:  U-LOC
Entity in Στοιχάδες : U-LOC | expected:  O
Entity in Ἰσθμοῦ : U-LOC | expected:  U-LOC
Entity in Γάγγου : U-LOC | expected:  O
Entity in Ἰλλυρίδα : U-LOC | expected:  U-LOC
Entity in λίμνης, : U-LOC | expected:  U-LOC
Entity in Εὐρώπης : U-LOC | expected:  U-LOC
Entity in Σαρδῶον : U-LOC | expected:  U-LOC
Entity in Εὐρώπης : U-LOC | expected:  U-LOC
Entity in Μιλήσιος : U-LOC | expected:  U-LOC
Entity in Σαρδὼ : U-LOC | expected:  U-LOC
Entity in ῾Ρόδον : U-LOC | expected:  O
Entity in Ὅροι : U-LOC | expected:  O
Entity in Ἐκλήθησαν : U-LOC | expected:  O
Entity in μακρὰ : U-LOC | expected:  O
Entity in Κανωβικοῦ : U-LOC | expected:  U-LOC
Entity in Σύρτεις : U-LOC | expected:  U-LOC
Entity in ↑ν, : U-LOC | expected:  O
Entity in Θοάντιον : U-LOC | expected:  U-LOC
Entity in ποταμοῦ : U-LOC | expected:  O
Entity in Λιβυκοῦ : U-LOC | expected:  U-LOC
Entity in Λιγυστικὸν : U-LOC | expected:  U-LOC
Entity in μικρᾶς : U-LOC | expected:  U-LOC